<h1><center>StationRank</center></h1>

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
import pandas as pd
import geopandas as gpd
import colorcet as cc
import contextily as ctx
import ipywidgets as widgets
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [3]:
from matplotlib.lines import Line2D
from adjustText import adjust_text
from mpl_toolkits.axes_grid1 import make_axes_locatable
from ipywidgets import interactive, Layout, VBox, AppLayout

In [4]:
base_dir = f"data/"
vector_dir = f"{base_dir}vectors/"

In [17]:
# Map tiles
source_simple = 'https://tile.jawg.io/b0bdce7b-036a-4842-a818-81c6af57f09c/{z}/{x}/{y}.png?access-token=zsakaer0HRCbNjEC6TExV32UzssAHDI34iKwJhdHi5hUa9v5xzanjWAtzOQVgCUt'
source_detail = 'https://tile.jawg.io/db400043-2505-4b3a-81c2-16b44c722134/{z}/{x}/{y}.png?access-token=zsakaer0HRCbNjEC6TExV32UzssAHDI34iKwJhdHi5hUa9v5xzanjWAtzOQVgCUt'
source_online = 'https://api.mapbox.com/styles/v1/ganagno/ckffylm1u0civ19ochci2gw4t/tiles/512/{Z}/{X}/{Y}?access_token=pk.eyJ1IjoiZ2FuYWdubyIsImEiOiJjanpreGRzdXAwbmJkM2dvOWdibjhzOXZjIn0.OJP0eGHOq7qEb9fLSq0wBw'

In [6]:
# Get vector files
vectors = []
files = sorted(os.listdir(vector_dir))
for file in (files):
    file = f'{vector_dir}{file}'
    vector = pd.read_csv(file)
    # Georeferenced locations by BPUIC
    vector = gpd.GeoDataFrame(vector,
                              geometry=gpd.points_from_xy(
                                  vector.E_WGS84,
                                  vector.N_WGS84,
                                  crs={'init': 'epsg:4326'}))
    vector['timestamp'] = vector['timestamp'].str.replace('_', '.')
    vectors.append(vector)

In [7]:
file = f'{base_dir}geolocations.csv'
geolocations = pd.read_csv(file)
# Georeferenced locations by BPUIC
geolocations = gpd.GeoDataFrame(geolocations,
                                geometry=gpd.points_from_xy(
                                    geolocations.E_WGS84,
                                    geolocations.N_WGS84,
                                    crs={'init': 'epsg:4326'}))

In [8]:
timestamps = list(pd.concat(vectors).timestamp.unique())

In [9]:
def myfmt(x, _):
    if x < 1:
        form =  '{0:+.3f}'.format(x)
    else:
        form =  '{0:+.1f}'.format(x)      
    return form

In [10]:
def mapper(vector, geolocations, column, cmap, norm, markersize, label):
    f, ax = plt.subplots(figsize=(15, 10))
    font = {'size': 12}
    plt.rc('font', **font)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="3%", pad=0.2)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    # Convert to Web Mercator
    geolocations = geolocations.to_crs(epsg=3857)
    geolocations.plot(ax=ax,
                      zorder=5,
                      markersize=markersize-5,
                      fc='grey',
                      ec='k',
                      lw=0.2)
    # Convert to Web Mercator
    vector = vector.to_crs(epsg=3857)
    vector.plot(ax=ax,
                cax=cax,
                zorder=10,
                column=column,
                cmap=cmap,
                norm=norm,
                markersize=markersize,
                ec='k',
                lw=0.2)
    cbar = f.colorbar(sm, cax=cax, format=ticker.FuncFormatter(myfmt))
    cbar.set_label(label)
    legend_elements = [
        Line2D([0], [0],
               marker='o',
               color='k',
               label='excluded',
               markerfacecolor='grey',
               markeredgewidth=0.6,
               markersize=10,
               lw=0)
    ]
    ax.legend(handles=legend_elements, loc='upper left')
    try:
        ctx.add_basemap(ax, attribution=False, source=source_simple, zoom=8)
    except:
        pass
    ax.set_axis_off()
    f.tight_layout(pad=1.)
    plt.show()
    plt.close()

In [11]:
def calibrate(vectors, column):
    if column == 'remoteness':
        cmap = cc.m_fire
        vmin = 20
        vmax = 100
        label = 'Cube-root of transpose mean first passage time'
    elif any(column in item for item in ['stationary', 'inflow' 'outflow']):
        cmap = cc.m_CET_L19
        vmin = 0.02
        vmax = 0.16
        label = 'Cube-root of the first eigenvector'
    elif column == 'cluster':
        cmap = 'turbo'
        vmin = -0.5
        vmax = 0.4
        label = 'Cube-root of the second eigenvector'
    elif column == 'fragility':
        cmap = cc.m_CET_L16
        vmin = 0.1
        vmax = 1
        label = 'Systemic fragility'
    elif column == 'influence':
        cmap = cc.m_CET_L19
        vmin = 0.1
        vmax = 1
        label = 'Cube-root of systemic influence'
    else:
        cmap = 'seismic'
        vmin = -80
        vmax = +80
        label = f'Effect on stationary distribution by a disruption at {column}'
    norm = plt.Normalize(vmin=vmin, vmax=vmax)
    return cmap, norm, label

In [12]:
def show_mapper(vectors=vectors, date=timestamps[0], column='remoteness'):
    cmap, norm, label = calibrate(vectors, column)
    markersize = 80

    return mapper(vectors[timestamps.index(date)], geolocations, column, cmap,
                  norm, markersize, label)

In [13]:
def bubbles(date=timestamps[0]):
    ind = timestamps.index(date)
    font = {'size': 12}
    plt.rc('font', **font)
    fig = plt.figure(figsize=(12, 7))
    plt.subplot(1, 1, 1)
    plt.grid(lw=.3, color='gray', linestyle=(0, (5, 5)), zorder=-1)
    ax = plt.gca()
    ax.set_xscale('linear')
    ax.set_yscale('linear')
    x = vectors[ind].fragility.values
    y = vectors[ind].influence.values
    z = vectors[ind].stationary.values
    s_exp = [((n)**3 * 1e4)**2 for n in z]
    ax.scatter(x,
               y,
               s=s_exp,
               lw=.4,
               marker='o',
               ec=(0, 0, 0, 1),
               fc=(0, 0, 1, 0.4),
               label='$r\propto\pi_i$',
               zorder=10)
    rule1 = vectors[ind].fragility.values > .65
    rule2 = vectors[ind].stationary.values > .1
    rule3 = vectors[ind].influence.values > .8

    subset = np.where(np.logical_or(np.logical_and(rule1, rule2), rule3))[0]
    texts = [
        ax.annotate(
            vectors[ind].HALTESTELLEN_NAME.values[i],
            xy=(vectors[ind].fragility.values[i] - 0.0,
                vectors[ind].influence.values[i] - 0.0),
        ) for i in subset
    ]
    adjust_text(texts)
    plt.xlabel('systemic fragility (absolute change > {}%)'.format(thresh))
    plt.ylabel('systemic influence (absolute change > {}%)'.format(thresh))
    font = {'size': 14.}
    plt.rc('font', **font)
    plt.tight_layout()
    leg = ax.legend(loc='best',
                    borderpad=2,
                    handletextpad=1.5,
                    framealpha=1,
                    shadow=False,
                    fancybox=True)
    leg.legendHandles[0].set_facecolor('white')
    leg.legendHandles[0].set_edgecolor('black')
    ax.autoscale(enable=True, axis='both', tight=False)
    plt.ylim(0.1, 1.1)
    plt.xlim(0.1, 1.1)
    plt.show()
    plt.close()

In [14]:
thresh = 1
date = widgets.Dropdown(options=list(timestamps), value=timestamps[0])
date.layout.width = '70%'
ui = widgets.VBox([date], layout=Layout(flex_flow='col wrap'))
ui.layout.align_items = 'flex-end'
ui.layout.padding = '.9em'
out = widgets.interactive_output(bubbles, {'date': date})
AppLayout(left_sidebar=ui, center=out, pane_widths=[1, 3, 1], align_items='top')

AppLayout(children=(VBox(children=(Dropdown(layout=Layout(width='70%'), options=('01.10.2019', '02.10.2019', '…

In [18]:
columns = ['remoteness', 'cluster', 'stationary', 'influence', 'fragility']
column = widgets.Dropdown(options=list((columns)), value=columns[-1])
column.layout.width = '70%'
date = widgets.Dropdown(options=list(timestamps), value=timestamps[0])
date.layout.width = '70%'
ui = widgets.VBox([column, date], layout=Layout(flex_flow='col wrap'))
ui.layout.align_items = 'flex-end'
ui.layout.padding = '.9em'
out = widgets.interactive_output(show_mapper, {'date': date, 'column': column})
AppLayout(left_sidebar=ui, center=out, pane_widths=[1, 3, 1], align_items='top')

AppLayout(children=(VBox(children=(Dropdown(index=4, layout=Layout(width='70%'), options=('remoteness', 'clust…

In [19]:
columns = vector.columns.values[15:-1]
column = widgets.Dropdown(options=list((columns)), value='Winterthur')
column.layout.width = '70%'
date = widgets.Dropdown(options=list(timestamps), value=timestamps[0])
date.layout.width = '70%'
ui = widgets.VBox([column, date], layout=Layout(flex_flow='col wrap'))
ui.layout.align_items = 'flex-end'
ui.layout.padding = '.9em'
out = widgets.interactive_output(show_mapper, {'date': date, 'column': column})
AppLayout(left_sidebar=ui, center=out, pane_widths=[1, 3, 1], align_items='top')

AppLayout(children=(VBox(children=(Dropdown(index=2, layout=Layout(width='70%'), options=('Landquart', 'Olten'…